In [ ]:
import torch
import torch.nn as nn
from torch.functional import F
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
from collections import Counter
import random
from IPython.display import clear_output
import math

In [ ]:
%load_ext autoreload
%autoreload 2

# Get Data

### The Critique of Pure Reason

In [ ]:
with open('reason.txt') as f:
    fulltext = f.read()

In [ ]:
start = fulltext.index('PREFACE TO THE FIRST EDITION 1781')
start

In [ ]:
end = fulltext.index('End of the Project Gutenberg EBook')
end

In [ ]:
fulltext = fulltext[start:end]

In [ ]:
fulltext = fulltext.lower()

In [ ]:
len(fulltext)

# Tokenization

In [ ]:
def tokenize(text):
    text = re.sub('_', '', text)
    text = re.sub('\d+', '', text)
    fulltoks = re.findall(r"[\w']+|[,.?;:]", text)
    return fulltoks

In [ ]:
fulltoks = tokenize(fulltext)

In [ ]:
vocab = [tok for tok,count in Counter(fulltoks).most_common()]

In [ ]:
len(vocab)

# Transforms

In [ ]:
from helpers import BaseTransform, TokTransform

In [ ]:
tok_tfm = TokTransform(vocab)

In [ ]:
data = tok_tfm.encode(fulltoks)
x = data[:-1]
y = data[1:]
assert len(x) == len(y)
assert len(data) == len(x)+1

# Chunking

In [ ]:
chunk_sz = 50

In [ ]:
n_chunks = math.floor(len(x) / chunk_sz)
n_chunks

In [ ]:
x = x[:n_chunks*chunk_sz]; x = x.chunk(n_chunks)
y = y[:n_chunks*chunk_sz]; y = y.chunk(n_chunks)

In [ ]:
assert(len(x) == len(y))

# Train / Val split

In [ ]:
def shuffle_same(x_set, y_set):
        "Shuffle both x_set and y_set, but keep them lined up with each other"
        zipped = list(zip(x_set, y_set))
        random.shuffle(zipped)
        return list(zip(*zipped))

In [ ]:
x,y = shuffle_same(x,y)

In [ ]:
len(x)

In [ ]:
cut = int(len(x) * .8)
cut

In [ ]:
range(cut), range(cut,len(x))

In [ ]:
x_train, x_val = x[:cut], x[cut:]
y_train, y_val = y[:cut], y[cut:]

In [ ]:
assert len(x_train) == len(y_train)
assert len(x_val) == len(y_val)

# Dataloading

In [ ]:
from helpers import DataLoader, DataLoaders

In [ ]:
bs = 16
dl_train = DataLoader(x_train, y_train, bs)
dl_val = DataLoader(x_val, y_val, bs)
dls = DataLoaders(dl_train, dl_val)

# Model

In [ ]:
from model import LangModel

In [ ]:
voc_sz = tok_tfm.count
emb_sz = 200
hid_sz = 300

In [ ]:
model = LangModel(voc_sz, emb_sz, hid_sz)

# Loss

In [ ]:
def LM_loss(preds, targ):
    preds = preds.view(-1, tok_tfm.count)
    targ = targ.view(-1)
    # don't include xxunk indices
    preds = preds[targ!=0]
    targ = targ[targ!=0]
    return F.cross_entropy(preds, targ)

# Optimizer

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=.001)

# Metrics (accuracy)

In [ ]:
def accuracy(preds, targ):
    # don't include xxunk indices
    preds = preds[targ!=0]
    targ = targ[targ!=0]
    nums = preds.argmax(dim=-1)
    return (nums == targ).float().mean()

# Learner

In [ ]:
from helpers import Learner

In [ ]:
learn = Learner(dls, model, LM_loss, opt, accuracy)

In [ ]:
learn.fit(3)

In [ ]:
learn.print_logs()

# Saving

In [ ]:
torch.save(learn.model, 'saves/model')

In [ ]:
import pickle

In [ ]:
pickle.dump(tok_tfm, open('saves/tok_tfm.p', 'wb'))